In [31]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import pandas as pd
import numpy as np

# For csv merging from folder
import os

# For parsing key-value pairs in dataframe
import ast

# For stats analysis
from scipy import stats

# For basket analysis
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# For Counter / Synergy
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact

In [33]:
# Set the path to the directory containing the CSV files
path = '/content/drive/MyDrive/ME 379M/Hackathon 2023/Data'

# Create an empty list to store the DataFrames
dfs = []

# Iterate through all files in the directory and check if the file is a CSV file
for file in os.listdir(path):
    if file.endswith('.csv'):
        # read the CSV file into a DataFrame and append it to the list
        df = pd.read_csv(os.path.join(path, file))
        dfs.append(df)

# Concatenate all DataFrames into one final DataFrame
battle_data_df = pd.concat(dfs, axis=0, ignore_index=True)

print(len(battle_data_df))

870486


In [34]:
print("Imported Length:", len(battle_data_df))

# Clean data by removing rows with empty cells
battle_data_df = battle_data_df.dropna()
print("Without N/A:", len(battle_data_df))

# Filter out for competitive gamemodes only
unique_values = ['PvP', 'pathOfLegend']
battle_data_df = battle_data_df[battle_data_df['type'].isin(unique_values)]
print("With Filtered Game Modes:", len(battle_data_df))

Imported Length: 870486
Without N/A: 673737
With Filtered Game Modes: 222703


In [35]:
sample_battle_data_df = battle_data_df.sample(n=50000,random_state=42)

In [36]:
pd.set_option('display.max_colwidth', 50) 

In [37]:
# Parsing card data for player 1
sample_battle_data_df['cards1_dict'] = sample_battle_data_df['cards1'].apply(lambda x: ast.literal_eval(x))
sample_battle_data_df['deck1'] = sample_battle_data_df['cards1_dict'].apply(lambda x: [a['name'] for a in x])
sample_battle_data_df['max_level_1'] = sample_battle_data_df['cards1_dict'].apply(lambda x: [a['maxLevel'] for a in x])
sample_battle_data_df['level_1'] = sample_battle_data_df['cards1_dict'].apply(lambda x: [a['level'] for a in x])

In [38]:
# Parsing card data for player 2
sample_battle_data_df['cards2_dict'] = sample_battle_data_df['cards2'].apply(lambda x: ast.literal_eval(x))
sample_battle_data_df['deck2'] = sample_battle_data_df['cards2_dict'].apply(lambda x: [a['name'] for a in x])
sample_battle_data_df['max_level_2'] = sample_battle_data_df['cards2_dict'].apply(lambda x: [a['maxLevel'] for a in x])
sample_battle_data_df['level_2'] = sample_battle_data_df['cards2_dict'].apply(lambda x: [a['level'] for a in x])

In [39]:
sample_battle_data_df.head()

,type,gameMode,crowns1,cards1,startingTrophies2,crowns2,cards2,cards1_dict,deck1,max_level_1,level_1,cards2_dict,deck2,max_level_2,level_2
202169,pathOfLegend,"{'id': 72000328, 'name': 'Ranked1v1_CrownRush'}",0,"[{'name': 'Mirror', 'id': 28000006, 'level': 9...",1656.0,1,"[{'name': 'Zap', 'id': 28000008, 'level': 14, ...","[{'name': 'Mirror', 'id': 28000006, 'level': 9...","[Mirror, Inferno Dragon, Arrows, Guards, Giant...","[9, 6, 14, 9, 9, 9, 9, 12]","[9, 6, 14, 9, 9, 9, 9, 12]","[{'name': 'Zap', 'id': 28000008, 'level': 14, ...","[Zap, Ice Golem, Three Musketeers, Miner, Batt...","[14, 12, 12, 6, 12, 14, 14, 12]","[14, 12, 12, 6, 12, 14, 14, 12]"
61152,PvP,"{'id': 72000201, 'name': 'Ladder_CrownRush'}",1,"[{'name': 'Ram Rider', 'id': 26000051, 'level'...",6849.0,3,"[{'name': 'Minion Horde', 'id': 26000022, 'lev...","[{'name': 'Ram Rider', 'id': 26000051, 'level'...","[Ram Rider, Goblin Barrel, Dark Prince, Valkyr...","[6, 9, 9, 12, 14, 14, 14, 6]","[6, 9, 9, 12, 14, 14, 14, 6]","[{'name': 'Minion Horde', 'id': 26000022, 'lev...","[Minion Horde, Golden Knight, Elixir Golem, Wi...","[14, 4, 12, 12, 9, 9, 14, 9]","[14, 4, 12, 12, 9, 9, 14, 9]"
161173,PvP,"{'id': 72000006, 'name': 'Ladder'}",1,"[{'name': 'Archers', 'id': 26000001, 'level': ...",6756.0,0,"[{'name': 'The Log', 'id': 28000011, 'level': ...","[{'name': 'Archers', 'id': 26000001, 'level': ...","[Archers, Bomb Tower, Goblins, Mighty Miner, M...","[14, 12, 14, 4, 6, 9, 6, 9]","[14, 12, 14, 4, 6, 9, 6, 9]","[{'name': 'The Log', 'id': 28000011, 'level': ...","[The Log, Wizard, Arrows, Rage, Bomber, Hog Ri...","[6, 12, 14, 9, 14, 12, 9, 9]","[6, 12, 14, 9, 14, 12, 9, 9]"
122044,PvP,"{'id': 72000006, 'name': 'Ladder'}",3,"[{'name': 'Bandit', 'id': 26000046, 'level': 5...",5530.0,0,"[{'name': 'Sparky', 'id': 26000033, 'level': 3...","[{'name': 'Bandit', 'id': 26000046, 'level': 5...","[Bandit, Valkyrie, Elite Barbarians, Musketeer...","[6, 12, 14, 12, 9, 9, 14, 14]","[5, 12, 13, 10, 8, 8, 14, 13]","[{'name': 'Sparky', 'id': 26000033, 'level': 3...","[Sparky, Firecracker, Zap, Tombstone, Electro ...","[6, 14, 14, 12, 6, 14, 9, 9]","[3, 10, 12, 8, 3, 12, 8, 6]"
682585,pathOfLegend,"{'id': 72000323, 'name': 'Ranked1v1'}",0,"[{'name': 'Miner', 'id': 26000032, 'level': 6,...",1539.0,1,"[{'name': 'Electro Giant', 'id': 26000085, 'le...","[{'name': 'Miner', 'id': 26000032, 'level': 6,...","[Miner, Archers, Mini P.E.K.K.A, Fireball, Ske...","[6, 14, 12, 12, 14, 9, 9, 12]","[6, 14, 12, 12, 14, 9, 9, 12]","[{'name': 'Electro Giant', 'id': 26000085, 'le...","[Electro Giant, Fire Spirit, Tornado, Phoenix,...","[9, 14, 9, 6, 9, 14, 4, 9]","[9, 14, 9, 6, 9, 14, 4, 9]"


In [40]:
# Importing card data to add elixir costs
card_stats_df = pd.read_csv('/content/drive/MyDrive/ME 379M/Hackathon 2023/Card Stats/card_info.csv')
card_stats_df.head()

,name,cost,category,type,hitpoints,damage,dps,count,total_hitpoints,total_damage,total_dps
0,Knight,3,Melee,Support,2339.0,267,222.0,1,2339,267,222.0
1,Archers,3,Ranged,Support,403.0,142,157.0,2,806,284,314.0
2,Goblins,2,Swarm,Support,267.0,159,144.0,4,1068,636,576.0
3,Giant,5,Tank,Win Con,5423.0,337,224.0,1,5423,337,224.0
4,P.E.K.K.A,7,Tank,Support,4982.0,1081,600.0,1,4982,1081,600.0


In [41]:
# Define a function to compute the elixir costs for a deck
def compute_elixir_costs(deck):
    card_costs = dict(zip(card_stats_df['name'], card_stats_df['cost']))
    elixir_costs = [card_costs.get(card, None) for card in deck if card not in [None, 'None']]
    return elixir_costs

# Compute the elixir costs for the 'deck1' column and replace it with the result
sample_battle_data_df['elixir_costs_1'] = sample_battle_data_df['deck1'].apply(compute_elixir_costs)

# Compute the elixir costs for the 'deck2' column and replace it with the result
sample_battle_data_df['elixir_costs_2'] = sample_battle_data_df['deck2'].apply(compute_elixir_costs)

# Drop rows with elixir_costs as None
sample_battle_data_df.dropna(subset=['elixir_costs_1', 'elixir_costs_2'], inplace=True)

In [42]:
# Drop key-value pair columns now that data is extracted
sample_battle_data_df.drop(['cards1','cards2','cards1_dict','cards2_dict'],axis=1,inplace=True)

In [43]:
# Function that merges two lists by making a list of tuple values of each list
def merge(list1, list2):
    merged_list = [(list1[i], list2[i]) for i in range(len(list1))]
    return merged_list

In [44]:
sample_battle_data_df['cards_level_1'] = [[x-y+14 for x,y in merge(sample_battle_data_df['level_1'].iloc[i],sample_battle_data_df['max_level_1'].iloc[i])] 
                                          for i in range(len(sample_battle_data_df))]
sample_battle_data_df['cards_level_2'] = [[x-y+14 for x,y in merge(sample_battle_data_df['level_2'].iloc[i],sample_battle_data_df['max_level_2'].iloc[i])] 
                                          for i in range(len(sample_battle_data_df))]

In [45]:
# Averaging card levels from each player's deck
sample_battle_data_df['avg_cards_level_1'] = [sum(x)/8 for x in sample_battle_data_df['cards_level_1']]
sample_battle_data_df['avg_cards_level_2'] = [sum(x)/8 for x in sample_battle_data_df['cards_level_2']]

# Calculate the card level differential (Player 2 Avg - Player 1 Avg)
sample_battle_data_df['avg_cards_level_diff'] = sample_battle_data_df['avg_cards_level_2']-sample_battle_data_df['avg_cards_level_1']

In [46]:
# Averaging elixir costs from each player's deck
sample_battle_data_df['avg_cost_1'] = [sum(x)/8 for x in sample_battle_data_df['elixir_costs_1']]
sample_battle_data_df['avg_cost_2'] = [sum(x)/8 for x in sample_battle_data_df['elixir_costs_2']]

# Calculate the elixir cost differential (Player 2 Avg - Player 1 Avg)
sample_battle_data_df['avg_cost_diff'] = sample_battle_data_df['avg_cost_2']-sample_battle_data_df['avg_cost_1']

In [47]:
# Changing crowns data into winning data for simplicity
sample_battle_data_df['crowns_won'] = sample_battle_data_df['crowns2'] - sample_battle_data_df['crowns1']
sample_battle_data_df['winner'] = 0
sample_battle_data_df['winner'].loc[sample_battle_data_df['crowns_won']>0] = 2
sample_battle_data_df['winner'].loc[sample_battle_data_df['crowns_won']<0] = 1

<ipython-input-47-0332e66fee50>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_battle_data_df['winner'].loc[sample_battle_data_df['crowns_won']>0] = 2
<ipython-input-47-0332e66fee50>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_battle_data_df['winner'].loc[sample_battle_data_df['crowns_won']<0] = 1


In [48]:
# Checking how often there is a tie
count = (sample_battle_data_df['crowns_won']==0).sum()
print(count)

2


In [49]:
# Drop ties from the sample
zero_rows = sample_battle_data_df.loc[sample_battle_data_df['crowns_won'] == 0]

# drop the zero rows from the dataframe
sample_battle_data_df.drop(zero_rows.index, inplace=True)

# Checking how often there is a tie
count = (sample_battle_data_df['crowns_won']==0).sum()
print(count)

0


In [50]:
sample_battle_data_df.drop(['max_level_1','level_1','max_level_2','level_2','cards_level_1','cards_level_2','elixir_costs_1','elixir_costs_2'],axis=1,inplace=True)

In [51]:
sample_battle_data_df.head()

,type,gameMode,crowns1,startingTrophies2,crowns2,deck1,deck2,avg_cards_level_1,avg_cards_level_2,avg_cards_level_diff,avg_cost_1,avg_cost_2,avg_cost_diff,crowns_won,winner
202169,pathOfLegend,"{'id': 72000328, 'name': 'Ranked1v1_CrownRush'}",0,1656.0,1,"[Mirror, Inferno Dragon, Arrows, Guards, Giant...","[Zap, Ice Golem, Three Musketeers, Miner, Batt...",14.000,14.00,0.000,3.750,4.125,0.375,1,2
61152,PvP,"{'id': 72000201, 'name': 'Ladder_CrownRush'}",1,6849.0,3,"[Ram Rider, Goblin Barrel, Dark Prince, Valkyr...","[Minion Horde, Golden Knight, Elixir Golem, Wi...",14.000,14.00,0.000,3.625,3.750,0.125,2,2
161173,PvP,"{'id': 72000006, 'name': 'Ladder'}",1,6756.0,0,"[Archers, Bomb Tower, Goblins, Mighty Miner, M...","[The Log, Wizard, Arrows, Rage, Bomber, Hog Ri...",14.000,14.00,0.000,3.000,3.625,0.625,-1,1
122044,PvP,"{'id': 72000006, 'name': 'Ladder'}",3,5530.0,0,"[Bandit, Valkyrie, Elite Barbarians, Musketeer...","[Sparky, Firecracker, Zap, Tombstone, Electro ...",13.125,11.25,-1.875,3.875,3.875,0.000,-3,1
682585,pathOfLegend,"{'id': 72000323, 'name': 'Ranked1v1'}",0,1539.0,1,"[Miner, Archers, Mini P.E.K.K.A, Fireball, Ske...","[Electro Giant, Fire Spirit, Tornado, Phoenix,...",14.000,14.00,0.000,3.750,3.750,0.000,1,2


In [52]:
sample_battle_data_df.to_csv("/content/drive/MyDrive/ME 379M/Hackathon 2023/battle_data_df.csv")

In [53]:
# Get full list of cards
full_list = list(set([card for row in sample_battle_data_df['deck2'] for card in row]))

In [54]:
# Make a dataframe of cards used
cards_df = pd.DataFrame(index=full_list)

In [55]:
def compute_cost(card_name):
    if card_name == None:
        return 0
    return card_stats_df.loc[card_stats_df['name'] == card_name, 'cost'].iloc[0]
cards_df['Cost'] = 0
for card_name in cards_df.index.tolist():
    cards_df.loc[card_name, 'Cost'] = compute_cost(card_name)
    if card_name == 'None':
        cards_df = cards_df.drop(cards_df[cards_df['name'] == card_name].index)

In [56]:
# Tally the uses of these by player 2
# Using only player 2 data for simplicity in calculating win rate
cards_df['Count'] = 0
for card in cards_df.index:
    count = 0
    for i in range(len(sample_battle_data_df)):
        if card in sample_battle_data_df['deck2'].iloc[i]:
            count+=1
    cards_df['Count'][card] = count

In [57]:
# Filter out wins for player 2 then count number of wins per card
cards_df['Wins'] = 0
win_df = sample_battle_data_df.loc[sample_battle_data_df['winner']==2].reset_index()
for card in cards_df.index:
    count = 0
    for i in range(len(win_df)):
        if card in win_df['deck2'].iloc[i]:
            count+=1
    cards_df['Wins'][card] = count

In [58]:
cards_df.head()

,Cost,Count,Wins
Mega Minion,3,1289,635
Witch,5,12852,6111
Arrows,3,13316,6289
Rascals,5,336,175
Flying Machine,4,899,439


In [59]:
cards_df['Use_Rate'] = round(cards_df['Count']/len(sample_battle_data_df), 3)
cards_df['Win_Rate'] = round(cards_df['Wins']/cards_df['Count'],3)

In [60]:
cards_df.head()


,Cost,Count,Wins,Use_Rate,Win_Rate
Mega Minion,3,1289,635,0.026,0.493
Witch,5,12852,6111,0.257,0.475
Arrows,3,13316,6289,0.266,0.472
Rascals,5,336,175,0.007,0.521
Flying Machine,4,899,439,0.018,0.488


In [62]:
cards_df.to_csv("/content/drive/MyDrive/ME 379M/Hackathon 2023/Card Stats/card_rates.csv")

In [63]:
def find_synergy_list(card_object, card_object_list, dataframe):
    # Grab every card except for the card itself
    search_list = full_list.copy()
    search_list.remove(card_object)
    result = pd.DataFrame(columns=['Synergy','Match_Count','Win_Count','Win_Rate','Match_Count_Without_Card','Win_Count_Without_Card','Win_Rate_Without_Card'])

    # Create empty lists to store information about each card in the search_list
    Synergy_card = []
    Match_Count = []
    Win_Count = []
    Match_Count_B = []
    Win_Count_B = []

    # For each card in the compiled search list:
    for synergy in search_list:
        wins = 0
        matches = 0
        wins_B = 0
        matches_B = 0

        # Get all rows of the DataFrame where player 2 has the synergy and the card_object
        p2 = dataframe.loc[(dataframe['deck2'].apply(lambda x: (synergy in x) & (card_object in x)))]
        # Count the number of wins and matches for player 2
        wins += len(p2.loc[p2['winner']==2])
        matches += len(p2)

        # Get all rows of the DataFrame where player 2 has the  card_object, but not the synergy
        p2B = dataframe.loc[(dataframe['deck2'].apply(lambda x: (synergy not in x) & (card_object in x)))]
        # Count the number of wins and matches for player 1 in this scenario
        wins_B += len(p2B.loc[p2B['winner']==1])
        matches_B += len(p2B)

        # Get all rows of the DataFrame where player 1 has the synergy and the card_object
        p1 = dataframe.loc[(dataframe['deck1'].apply(lambda x: (synergy in x) & (card_object in x)))]
        # Count the number of wins and matches for player 1
        wins += len(p1.loc[p1['winner']==1])
        matches += len(p1)

        # Get all rows of the DataFrame where player 1 has the  card_object, but not the synergy
        p1B = dataframe.loc[(dataframe['deck1'].apply(lambda x: (synergy not in x) & (card_object in x)))]
        # Count the number of wins and matches for player 2 in this scenario
        wins_B += len(p1B.loc[p1B['winner']==2])
        matches_B += len(p1B)

        # Append data to lists 
        Synergy_card.append(synergy)
        Win_Count.append(wins)
        Match_Count.append(matches)
        Match_Count_B.append(matches_B)
        Win_Count_B.append(wins_B)

    # Create result dataframe columns
    result['Synergy'] = Synergy_card
    result['Match_Count'] = Match_Count
    result['Win_Count'] = Win_Count
    result['Win_Rate'] = result['Win_Count']/result['Match_Count']
    result['Match_Count_Without_Card'] = Match_Count_B
    result['Win_Count_Without_Card'] = Win_Count_B
    result['Win_Rate_Without_Card'] = result['Win_Count_Without_Card']/result['Match_Count_Without_Card']

    # Normalize columns without card
    result['Match_Count_Without_Card'] = result['Match_Count']
    result['Win_Count_Without_Card_f'] = result['Match_Count_Without_Card']*result['Win_Rate_Without_Card']
    result['Win_Count_Without_Card'] = round(result['Win_Count_Without_Card_f'],0)
    result.drop(['Win_Count_Without_Card_f'],axis=1,inplace=True)
    return result

In [65]:
print(full_list)

['Mega Minion', 'Witch', 'Arrows', 'Rascals', 'Flying Machine', 'Zappies', 'Elixir Golem', 'Guards', 'Royal Giant', 'Lumberjack', 'Executioner', 'Phoenix', 'Valkyrie', 'Fireball', 'Goblins', 'Dark Prince', 'Skeletons', 'Minion Horde', 'Miner', 'Goblin Hut', 'Electro Dragon', 'Fisherman', 'Ram Rider', 'Barbarian Barrel', 'Bomber', 'Hog Rider', 'Night Witch', 'Wizard', 'Cannon', 'Minions', 'Archer Queen', 'Lava Hound', 'Bandit', 'Rocket', 'Wall Breakers', 'Barbarians', 'Mighty Miner', 'Goblin Cage', 'Mother Witch', 'Royal Recruits', 'Cannon Cart', 'Giant', 'Fire Spirit', 'Hunter', 'Electro Spirit', 'Skeleton Barrel', 'Royal Delivery', 'Electro Giant', 'Earthquake', 'Ice Wizard', 'Goblin Gang', 'Elixir Collector', 'Electro Wizard', 'Golden Knight', 'Prince', 'The Log', 'P.E.K.K.A', 'Tornado', 'Firecracker', 'Skeleton Army', 'Mirror', 'Furnace', 'Mini P.E.K.K.A', 'Giant Snowball', 'Battle Ram', 'Spear Goblins', 'Royal Ghost', 'Ice Spirit', 'Giant Skeleton', 'Golem', 'Bats', 'Rage', 'Goblin

In [66]:
for card_name in full_list:
    card_lookup = card_name
    synergy_table = find_synergy_list(card_lookup, full_list, sample_battle_data_df)
    synergy_table.to_csv("/content/drive/MyDrive/ME 379M/Hackathon 2023/Synergy Tables/" + card_name + ".csv")

In [68]:
import os
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact, chi2_contingency

# Set paths for input and output folders
input_folder = "/content/drive/MyDrive/ME 379M/Hackathon 2023/Synergy Tables"
output_folder = "/content/drive/MyDrive/ME 379M/Hackathon 2023/Synergy Tables 2"

# Get list of CSV files in the input folder
csv_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.csv')]

# Loop through each CSV file
for csv_file in csv_files:
    # Load CSV file as DataFrame
    df = pd.read_csv(csv_file)
    
    # Perform the operation you provided
    p_values = []
    synergy_table = df.dropna()
    for champion in list(synergy_table['Synergy']):
        a = int(synergy_table['Win_Count'].loc[synergy_table['Synergy']==champion])
        b = int(synergy_table['Win_Count_Without_Card'].loc[synergy_table['Synergy']==champion])
        c = int(synergy_table['Match_Count'].loc[synergy_table['Synergy']==champion] - synergy_table['Win_Count'].loc[synergy_table['Synergy']==champion])
        d = int(synergy_table['Match_Count_Without_Card'].loc[synergy_table['Synergy']==champion] - synergy_table['Win_Count_Without_Card'].loc[synergy_table['Synergy']==champion])
        ct = np.array([[a,b], [c,d]])
        if int(c+a) <= 50:
            p_values.append(fisher_exact(ct,alternative='two-sided')[1])    
        else:
            p_values.append(chi2_contingency(ct)[1])
    synergy_table['P-Value'] = p_values
    synergy_table.drop(['Match_Count','Win_Count','Match_Count_Without_Card','Win_Count_Without_Card'],axis=1,inplace=True)
    synergy_table = synergy_table.sort_values(by=['P-Value'],ascending=True).loc[synergy_table['Win_Rate']>synergy_table['Win_Rate_Without_Card']]
    
    # Export the resulting DataFrame to a CSV file in the output folder
    output_file = os.path.join(output_folder, os.path.basename(csv_file))
    synergy_table.to_csv(output_file, index=False)


<ipython-input-68-d17fc46cb518>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synergy_table['P-Value'] = p_values
<ipython-input-68-d17fc46cb518>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synergy_table.drop(['Match_Count','Win_Count','Match_Count_Without_Card','Win_Count_Without_Card'],axis=1,inplace=True)
<ipython-input-68-d17fc46cb518>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/